# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Setup and Import

In [1]:
# ! pip install -q "datasets==2.15.0"

In [2]:
pip install -U transformers accelerate peft bitsandbytes scikit-learn

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 60.6 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.9/504.9 kB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 24.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 98.3 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 93.6 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.8/485.8 kB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.0/888.0 MB 1.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.4/308.4 kB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 96.5 MB/s eta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 20.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 1.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.2/287.2 MB 4.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 17.2 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.21.4
    Uninstalling huggingface-hub-0.21.4:
      Successfully uninstalled huggingface-hub-0.21.4
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-l

In [3]:
import numpy as np
import torch
import os
import shutil

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    # BitsAndBytesConfig,
    TrainingArguments,
    Trainer, 
    DataCollatorWithPadding
)

from peft import LoraConfig, get_peft_model

from sklearn.metrics import accuracy_score, f1_score

root_dir = "./tmp"
# root_dir = "./../../../../data/GenAI/02_genai_fundamentals/project2/results"

training_artifacts_dir = 'training_artifacts'
archive_zip = 'archive'

foundation_model_name = "distilbert-base-uncased"

foundation_model_path = os.path.join(root_dir, training_artifacts_dir, "foundation_model", "model")
foundation_model_output_path = os.path.join(root_dir, training_artifacts_dir, "foundation_model", "output")


lora_model_path = os.path.join(root_dir, "lora_model", "model")
lora_model_output_path = os.path.join(root_dir, "lora_model", "output")

qlora_model_path = os.path.join(root_dir, "qlora_model", "model")
qlora_model_output_path = os.path.join(root_dir, "qlora_model", "output")
                                
                                
batch_size = 16
train_epochs = 2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


## Clean up

In [4]:
# Check if the folder exists before attempting to delete
folder_to_delete = root_dir
if os.path.exists(folder_to_delete):
    try:
        shutil.rmtree(folder_to_delete)
        print(f"Folder '{folder_to_delete}' and its contents deleted successfully.")
    except OSError as e:
        print(f"Error: {folder_to_delete} : {e.strerror}")
else:
    print(f"Folder '{folder_to_delete}' does not exist.")

Folder './tmp' and its contents deleted successfully.


## Load Data

### Load complete dataset

In [5]:
# The sms_spam dataset only has a train split, so we use the train_test_split method to split it into train and test
dataset = load_dataset("sms_spam", split="train").train_test_split(
    test_size=0.2, shuffle=True, seed=23
)

splits = ["train", "test"]

# View the dataset characteristics
dataset["train"]



Generating train split:   0%|          | 0/5574 [00:00<?, ? examples/s]

Dataset({
    features: ['sms', 'label'],
    num_rows: 4459
})

### Load subset for testing

In [6]:
# dataset = load_dataset("sms_spam", split="train")
# dataset = dataset.select(range(3))

# dataset = dataset.train_test_split(test_size=0.2, shuffle=True, seed=23)


# splits = ["train", "test"]

# # View the dataset characteristics
# dataset["train"]

In [7]:
# Inspect the first example
dataset["train"][0]

{'sms': 'Had your mobile 10 mths? Update to the latest Camera/Video phones for FREE. KEEP UR SAME NUMBER, Get extra free mins/texts. Text YES for a call\n',
 'label': 1}

## Pre-process dataset

In [8]:
tokenizer = AutoTokenizer.from_pretrained(foundation_model_name)

# Let's use a lambda function to tokenize all the examples
tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["sms"], truncation=True), batched=True
    )


# Inspect the available columns in the dataset
tokenized_dataset["train"]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/4459 [00:00<?, ? examples/s]

Map:   0%|          | 0/1115 [00:00<?, ? examples/s]

Dataset({
    features: ['sms', 'label', 'input_ids', 'attention_mask'],
    num_rows: 4459
})

In [9]:
# Inspect the available columns in the dataset
tokenized_dataset["train"]

Dataset({
    features: ['sms', 'label', 'input_ids', 'attention_mask'],
    num_rows: 4459
})

## Helper Functions

In [10]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

### Train, Evaluate and Save Model

In [11]:
def TrainAndValidate(model, model_output_path, model_path, tokenizer, is_foundation_model=False):
    model.to(device)
    
    # Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
    trainer = Trainer(
        model=model,
        args=TrainingArguments(
            output_dir=model_output_path,
            # Set the learning rate
            learning_rate=2e-5,
            # Set the per device train batch size and eval batch size
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            # Evaluate and save the model after each epoch
            # evaluation_strategy="epoch",
            eval_strategy="epoch",
            save_strategy="epoch",
            num_train_epochs=train_epochs,
            weight_decay=0.01,
            load_best_model_at_end=True,
        ),
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["test"],
        tokenizer=tokenizer,
        data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
        compute_metrics=compute_metrics,
    )

    trainer.train()
    
    
    model.to(device)
    trainer.evaluate()
    
    print("saving model", model_path)
    model.save_pretrained(model_path)
    
    # Save tokenizer only for foundation model
    if is_foundation_model:
        tokenizer.save_pretrained(model_path)

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [12]:
foundation_model = AutoModelForSequenceClassification.from_pretrained(
    foundation_model_name,
    num_labels=2,
    id2label={0: "not spam", 1: "spam"},
    label2id={"not spam": 0, "spam": 1},
)

# Unfreeze all the model parameters.
# Hint: Check the documentation at https://huggingface.co/transformers/v4.2.2/training.html
for param in foundation_model.parameters():
    param.requires_grad = True

# print(foundation_model)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Train, Evaluate and Save Foundation Model

In [13]:
# TrainAndValidate(foundation_model, foundation_model_output_path, foundation_model_path)
TrainAndValidate(foundation_model, foundation_model_output_path, 
                 foundation_model_path, tokenizer, is_foundation_model=True)

/tmp/ipykernel_31/1553735947.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.048811,0.987444
2,0.054800,0.054236,0.987444


saving model ./tmp/training_artifacts/foundation_model/model


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

### Load from Saved Foundation model and Tokenizer

In [14]:
foundation_model = AutoModelForSequenceClassification.from_pretrained(foundation_model_path)
tokenizer = AutoTokenizer.from_pretrained(foundation_model_path)

In [15]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_lin", "k_lin", "v_lin"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS"
)

lora_model = get_peft_model(foundation_model, lora_config)

# lora_model.print_trainable_parameters()

### Train, Evaluate and Save Fine-Tuned Model

In [16]:
# TrainAndValidate(lora_model, lora_model_output_path, lora_model_path)
TrainAndValidate(lora_model, lora_model_output_path,
                 lora_model_path, tokenizer, is_foundation_model=False)

/tmp/ipykernel_31/1553735947.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.050074,0.987444
2,0.021700,0.050125,0.988341


saving model ./tmp/lora_model/model


###  ⚠️ IMPORTANT ⚠️

Due to workspace storage constraints, you should not store the model weights in the same directory but rather use `/tmp` to avoid workspace crashes which are irrecoverable.
Ensure you save it in /tmp always.

In [17]:
# Saving the model
#Already done in above steps

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

## Perform Inference

In [19]:
def evaluate_model(model, dataset, tokenizer):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    preds, labels = [], []

    for i in range(0, len(dataset), batch_size):
        batch = dataset[i : i + batch_size]

        # Tokenize this batch of text dynamically with padding + truncation
        inputs = tokenizer(
            batch["sms"],
            truncation=True,
            padding=True,
            return_tensors="pt",
            max_length=128
        ).to(device)

        with torch.no_grad():
            outputs = model(**inputs)

        preds += torch.argmax(outputs.logits, dim=1).cpu().tolist()
        labels += batch["label"]

    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds)
    return acc, f1

In [20]:
eval_dataset=tokenized_dataset["test"]

In [21]:
foundation_acc, foundation_f1 = evaluate_model(foundation_model, eval_dataset, tokenizer)
lora_acc, lora_f1 = evaluate_model(lora_model, eval_dataset, tokenizer)

In [22]:
print(f"Foundation Model - Accuracy: {foundation_acc:.4f}, F1: {foundation_f1:.4f}")
print(f"Lora Model - Accuracy: {lora_acc:.4f}, F1: {lora_f1:.4f}")

Foundation Model - Accuracy: 0.9874, F1: 0.9507
Lora Model - Accuracy: 0.9874, F1: 0.9507
